In [2]:
import numpy as np
import xarray as xr
import netCDF4 as nc
import glob
import datetime

In [ ]:
# regridding GCM data to 1 degree resolution
# loading ACCESS-CM2
# merging the data
scen = ["ssp126","ssp245","ssp370","ssp585"]
for s in range(0,len(scen)):
    path1 = "Z:/CMIP6/ACCESS-CM2/"+scen[s]+"/pr/pr_day_ACCESS-CM2_"+scen[s]+"_r1i1p1f1_gn_20150101-20641231.nc"
    path2 = "Z:/CMIP6/ACCESS-CM2/"+scen[s]+"/pr/pr_day_ACCESS-CM2_"+scen[s]+"_r1i1p1f1_gn_20650101-21001231.nc"
    comb = xr.open_dataset(path1)
# creating a new xarray for storing the unit edited precipitation in mm 
# multiplying by 24*3600 
#pr.sel(time='1960-06-15')
    time = comb.time
    lat = comb.lat
    lon = comb.lon
    pr2 = comb.pr*86400 # unit conversion if required
# new xarray
    edited = xr.DataArray(pr2, dims = ['time','lat','lon'], coords={'time':time,'lat':lat,'lon':lon,'pr':pr2})

# subsetting the xarray
#subset = edited.sel(time=slice('2031-01-01', '2064-12-31'), lon=slice(229,307), lat=slice(18, 52))
    subset = edited.sel(time=slice('2031-01-01', '2064-12-31'), lon=slice(229,307), lat=slice(18, 52))
# regrid the data to 1 degree resolution
# for any required resolution, create a mesh of the required resolution
# remap the nc files to 1 degree resolution using linear interpolation
    regrid_lat = np.arange(19.5,50.5,1)
    regrid_lon = np.arange(229.5,305.5,1)
# regrid datasets
    gcm_regrid =  subset.interp(lat=regrid_lat,lon=regrid_lon,method='linear')
# edit the path with complete file name
    path = 'H:/nanditha/CMIP6_decadal/ERA5-LAND/clipped/tp_mm_'+str(y)+'_region'+str(r)+'.nc'
# saving as nc files
    gcm_regrid.to_netcdf(path)
# writing the regridded data as separate text files for each grid
    for i in range(0,len(gcm_regrid.lat)):
        
        for j in range(0,len(gcm_regrid.lon)):
        # saving the files
        # one text file for each lat and lon
            np.savetxt('H:/nanditha/GCM/ACCESS-CM2/'+scen[s]+'/near/'+'precp_'+str(gcm_regrid.lat[i].to_numpy())+'_'+str(gcm_regrid.lon[j].to_numpy())+'.txt',gcm_regrid.pr[:,i,j].to_numpy(),fmt ='%.5f')
    comb.close()
    edited.close()
    subset.close()
    gcm_regrid.close()